In [423]:
#Basic Imports
import pandas as pd
import numpy as np
import matplotlib as plt
import statsmodels.api as sm
import seaborn as sns
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,summarize)
from sklearn.model_selection import train_test_split

In [424]:
#Classifying Imports
from ISLP import confusion_table
from ISLP.models import contrast
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OrdinalEncoder

In [425]:
Default = load_data("Default")
Default.head()

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879


In [426]:
enc = OrdinalEncoder(categories = [["No","Yes"]])
Default["student_encoded"] = enc.fit_transform(Default[["student"]])

In [427]:
default_train,default_test = train_test_split(Default,test_size=2000)

In [428]:
allvars = Default.columns.drop(["student","default"])
modelspec = MS(allvars)
X_train  = modelspec.fit_transform(default_train)
Y_train = default_train.default == "Yes"
model = sm.GLM(Y_train,X_train,family=sm.families.Binomial())
results = model.fit()
print(results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                default   No. Observations:                 8000
Model:                            GLM   Df Residuals:                     7996
Model Family:                Binomial   Df Model:                            3
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -617.88
Date:                Wed, 30 Apr 2025   Deviance:                       1235.8
Time:                        13:17:19   Pearson chi2:                 5.37e+03
No. Iterations:                     9   Pseudo R-squ. (CS):             0.1283
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
intercept         -10.9352      0.560    -

In [429]:

x_test = modelspec.fit_transform(default_test)
probs = results.predict(exog =  x_test)

In [430]:
labels = np.array(["No"]*2000)
labels[probs > 0.5] = "Yes"
labels = np.char.replace(labels,"Ye","Yes")
confusion_table(labels,default_test.default)

Truth,No,Yes
Predicted,,
No,1926,46
Yes,7,21


In [431]:
error = (45+12)/2000 * 100
print(f"Validation Set Test Error Rate {error}")

Validation Set Test Error Rate 2.85


We got an initial Validation Set Test Error Rate of 2.85% without the use of Student as a predictor. When we added it, the test error remained the same. 

Using Summarize, we saw that our standard errors for balance and income were pretty much 0 and 9x10e^-6. We'll now attempt to compute the same values using the bootstrap

Problem 7.

In Sections 5.1.2 and 5.1.3, we saw that the cross_validate() function
can be used in order to compute the LOOCV test error estimate.
Alternatively, one could compute those quantities using just sm.GLM()
and the predict() method of the ftted model within a for loop. You
will now take this approach in order to compute the LOOCV error
for a simple logistic regression model on the Weekly data set. Recall
that in the context of classifcation problems, the LOOCV error is
given in (5.4)

Fit a logistic regression model that predicts Direction using Lag1
and Lag2.
    (b) Fit a logistic regression model that predicts Direction using Lag1
    and Lag2 using all but the frst observation.
    (c) Use the model from (b) to predict the direction of the frst observation. You can do this by predicting that the frst observation
    will go up if P(Direction = "Up"|Lag1, Lag2) > 0.5. Was this
    observation correctly classifed?
    (d) Write a for loop from i = 1 to i = n, where n is the number of
    observations in the data set, that performs each of the following
    steps
        i. Fit a logistic regression model using all but the ith observation to predict Direction using Lag1 and Lag2.
        ii. Compute the posterior probability of the market moving up
        for the ith observation.
        iii. Use the posterior probability for the ith observation in order
        to predict whether or not the market moves up.
        iv. Determine whether or not an error was made in predicting
        the direction for the ith observation. If an error was made,
        then indicate this as a 1, and otherwise indicate it as a 0.

In [432]:
weekly = load_data("Weekly")
weekly.head()
Row = weekly.iloc[[0]]
weekly.drop(0)
Direction = Row["Direction"]
Row = Row.drop(["Today","Year","Lag3","Lag4","Lag5","Direction","Volume"],axis = 1)
print(Direction.values[0])

Down


In [433]:
Allvars = weekly.columns.drop(["Today","Year","Lag3","Lag4","Lag5","Direction","Volume"])


In [434]:
modelspec = MS(Allvars)
X = modelspec.fit_transform(weekly)
Y = weekly.Direction == "Up"
model = sm.GLM(Y,X,family=sm.families.Binomial())
results = model.fit()
print(results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:              Direction   No. Observations:                 1089
Model:                            GLM   Df Residuals:                     1086
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -744.11
Date:                Wed, 30 Apr 2025   Deviance:                       1488.2
Time:                        13:17:19   Pearson chi2:                 1.09e+03
No. Iterations:                     4   Pseudo R-squ. (CS):           0.007303
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.2212      0.061      3.599      0.0

In [435]:
row_test = modelspec.fit_transform(Row)
prob = results.predict(row_test)
labels = np.array(["Down"])
labels[prob > 0.5] = "Up"


This observation  was not correctly classified 

In [436]:
def LOOCV():
    weekly = load_data("Weekly")
    result = []
    for i in range(len(weekly) - 1):
        Row = weekly.iloc[[i]]
        weekly.drop(0)
        Direction = Row["Direction"]
        Row = Row.drop(["Today","Year","Lag3","Lag4","Lag5","Direction","Volume"],axis = 1)
        Allvars = weekly.columns.drop(["Today","Year","Lag3","Lag4","Lag5","Direction","Volume"])
        modelspec = MS(Allvars)
        X = modelspec.fit_transform(weekly)
        Y = weekly.Direction == "Up"
        model = sm.GLM(Y,X,family=sm.families.Binomial())
        results = model.fit()
        row_test = modelspec.fit_transform(Row)
        prob = results.predict(row_test)
        labels = np.array(["Down"])
        labels[prob > 0.5] = "Up"
        if labels[0] != Direction.values[0]:
            result.append(1)
        else:
            result.append(0)
    
    return result


results_from_loocv = LOOCV()

In [438]:
errors = 0
for num in results_from_loocv:
    if num == 1:
        errors +=1 

test_error = errors/len(results_from_loocv) * 100
print(f"Test error from LOOCV {test_error} %")

Test error from LOOCV 44.48529411764706 %


This model is terrible. 